In [ ]:
!pip install pgmpy

In [2]:
import pgmpy
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

In [3]:
diagnosis_model = BayesianModel([('Disease', 'Fever'),
                             ('Disease', 'Cough'),
                             ('Disease', 'Diarrhea')])

In [4]:
disease_cpd = TabularCPD(
                variable = 'Disease',
                variable_card = 3,
                values = [[0.5], [0.3], [0.2]])

In [5]:
fever_cpd = TabularCPD(
                variable = 'Fever',
                variable_card = 2,
                values = [[0.99, 0.02, 0.01],
                          [0.01, 0.98, 0.99]],
                evidence = ['Disease'],
                evidence_card = [3])

cough_cpd = TabularCPD(
                variable = 'Cough',
                variable_card = 2,
                values = [[0.99, 0.05, 0.9],
                          [0.01, 0.95, 0.1]],
                evidence = ['Disease'],
                evidence_card = [3])

diarrhea_cpd = TabularCPD(
                variable = 'Diarrhea',
                variable_card = 2,
                values = [[0.95, 0.8, 0.02],
                          [0.05, 0.2, 0.98]],
                evidence = ['Disease'],
                evidence_card = [3])

In [6]:
diagnosis_model.add_cpds(disease_cpd, fever_cpd, cough_cpd, diarrhea_cpd)

In [7]:
diagnosis_model.get_cpds()

[<TabularCPD representing P(Disease:3) at 0x7f099b53cda0>,
 <TabularCPD representing P(Fever:2 | Disease:3) at 0x7f099b4f95c0>,
 <TabularCPD representing P(Cough:2 | Disease:3) at 0x7f099b4f9588>,
 <TabularCPD representing P(Diarrhea:2 | Disease:3) at 0x7f099b4f9550>]

In [8]:
diagnosis_model.active_trail_nodes('Cough')

{'Cough': {'Cough', 'Diarrhea', 'Disease', 'Fever'}}

In [9]:
diagnosis_model.local_independencies('Cough')

(Cough _|_ Fever, Diarrhea | Disease)

In [10]:
diagnosis_model.get_independencies()

(Fever _|_ Cough, Diarrhea | Disease)
(Fever _|_ Diarrhea | Cough, Disease)
(Fever _|_ Cough | Diarrhea, Disease)
(Cough _|_ Fever, Diarrhea | Disease)
(Cough _|_ Diarrhea | Fever, Disease)
(Cough _|_ Fever | Diarrhea, Disease)
(Diarrhea _|_ Cough, Fever | Disease)
(Diarrhea _|_ Fever | Cough, Disease)
(Diarrhea _|_ Cough | Fever, Disease)

In [11]:
from pgmpy.inference import VariableElimination

In [12]:
diagnosis_infer = VariableElimination(diagnosis_model)

In [20]:
#Probability of diseases when cough is true.
prob_disease = diagnosis_infer.query(
                                     variables = ['Disease', 'Fever'],
                                     evidence = {'Cough': 1},
                                     joint=False)
print(prob_disease['Disease'])

Eliminating: Diarrhea: 100%|██████████| 1/1 [00:00<00:00, 579.88it/s]

+------------+----------------+
| Disease    |   phi(Disease) |
+============+================+
| Disease(0) |         0.0161 |
+------------+----------------+
| Disease(1) |         0.9194 |
+------------+----------------+
| Disease(2) |         0.0645 |
+------------+----------------+


In [21]:
diagnosis_infer.map_query(['Disease'])

Eliminating: Diarrhea: 100%|██████████| 3/3 [00:00<00:00, 482.05it/s]


{'Disease': 0}